In [1]:
# Multiplication of large numbers using polynomials
# This method has a cost in n.ln(n).ln(ln(n)) multiplying large numbers, while traditional multiplication has a n² cost,
# and Karatsuba algorithm is in n^1.58
# Ideas: 
# - number can be converted into a polynomial and back: 315 <-> 3x²+x5, with carry handling when converting polynomial -> number
# - a polynomial of degree n is uniquely defined by its value in (n+1) points. Lagrange interpolation converts (n+1) couples (x,y) into a polynomial
# - multiplication of polynomials can be done by multiplying values in several points: [PQ](x) = P(x).Q(x)
# - evaluation of a polynomial on (n+1) roots of unity can be done in n.ln(n) (Horner method is in n, and doing it on 2n values has a complexity of n²)
#
# 2018-08-13    PV

In [2]:
import numpy as np
from scipy.interpolate import lagrange
import math

In [3]:
# Simple polynomial test
p = np.poly1d([1,-3,5,2])
print(p)
print(p.c)
print(p(2), np.polyval(p, 2))

3     2
1 x - 3 x + 5 x + 2
[ 1 -3  5  2]
8 8


In [4]:
# Simple Lagrange interpolation test
xt = np.array([0, 1, 2])
yt = xt**2-5*xt+6     # Secret polynomial
p0 = lagrange(xt, yt)
print(p0)

2
1 x - 5 x + 6


In [5]:
# Direct multiplication using numpy polynomial multiplication
# Build polynomials for 123 and 245
p1 = np.poly1d([1,2,3])
p2 = np.poly1d([2,4,5])
p3a = np.polymul(p1,p2)
print(p3a)

4     3      2
2 x + 8 x + 19 x + 22 x + 15


In [6]:
# Normalization function to convert back a polynomial into a number, propagating carry
def polytonum(p):
    n = ""
    r = 0
    for i in reversed(p.c):
        assert abs(i.imag)<1e-6       # Safeguard, in this code, complex part of p coefficients must be almost nul
        assert abs(i.real - int(i.real+1e-6))<1e-6    # and real part must be integers
        r, d = divmod(int(i.real+1e-6)+r, 10)
        n = str(d) + n
    if r>0:
        n = str(r) + n
    return int(n)

print(polytonum(p3a))
print(123*245)      # Verification

30135
30135


In [7]:
# Alternate method to multiply polynomials, evaluate p1 and p2 in len(p1)+len(p2) points, multiply the results,
# then use lagrange interpolation to find associated polynomial
x = np.array([1,2,3,4,5,6])
y1 = p1(x)
y2 = p2(x)
y3 = y1*y2
p3b = lagrange(x, y3)
print(p3b)
print(polytonum(p3b))

4     3      2
2 x + 8 x + 19 x + 22 x + 15
30135


In [8]:
# Compute roots of unity.  Returns 1 as the last root.
def nthRootsOfUnity1(n):
    return np.exp(2j * np.pi / n * np.arange(1, n+1))

# Test, should return [j -1 -j 1]
print(nthRootsOfUnity1(4))

[ 6.1232340e-17+1.0000000e+00j -1.0000000e+00+1.2246468e-16j
 -1.8369702e-16-1.0000000e+00j  1.0000000e+00-2.4492936e-16j]


In [9]:
# Example of solving using complex roots of unity
# Skip last root since it's always 1
xc = nthRootsOfUnity1(7)[:-1]
y1c = p1(xc)
y2c = p2(xc)
y3c = y1c*y2c
p3c = lagrange(xc, y3c)
print(p3c)
print(polytonum(p3c))

5                    4                    3
1.776e-15j x + (2 + 7.105e-15j) x + (8 + 5.329e-15j) x
                      2
 + (19 + 3.553e-15j) x + (22 + 1.332e-15j) x + (15 + 2.22e-16j)
30135


In [10]:
# Helper, compare lists of complexes, sometimes difficult to read and compare on screen
def cmplc(l1, l2):
    if len(l1)!=len(l2):
        return False, "Different length"
    for i in range(len(l1)):
        if abs(l1[i]-l2[i])>1e-6: return False, "Different values at index "+str(i)
    return True, ""

In [11]:
# Now for discrete FFT optimization style, we need to compute n values where n+1 is a power of 2.
n = 7
x1 = nthRootsOfUnity1(n+1) 
x = x1[:-1]
print("x=", x)

p = np.poly1d([1,2,3,4,5,6,7,8])
print("p=\n",p, sep='')

# Recursive evaluation, posing p(x) = x*r(x²)+q(x²)
# q = polynomial(coefficients of even powers of p), contains constant coefficient (power 0)
# r = polynomial(coefficients of odd powers of p)
if len(p.c)%2==1:
    q = np.poly1d(p.c[::2])
    r = np.poly1d(p.c[1::2])
else:
    q = np.poly1d(p.c[1::2])
    r = np.poly1d(p.c[::2])
print("q=\n",q,sep='')
print("r=\n",r,sep='')

# xs contains the squares of x, taking every other values
xs = []
for i in range(n):
    xs.append(x1[(2*i+1)%(n+1)])
print(cmplc(xs, x*x))    # Verify
print()

e0 = p(x)
e1 = x*r(xs)+q(xs)
print("p(x)=", e0)
print(cmplc(e0, e1))     # Verify


x= [ 7.07106781e-01+7.07106781e-01j  6.12323400e-17+1.00000000e+00j
 -7.07106781e-01+7.07106781e-01j -1.00000000e+00+1.22464680e-16j
 -7.07106781e-01-7.07106781e-01j -1.83697020e-16-1.00000000e+00j
  7.07106781e-01-7.07106781e-01j]
p=
   7     6     5     4     3     2
1 x + 2 x + 3 x + 4 x + 5 x + 6 x + 7 x + 8
q=
   3     2
2 x + 4 x + 6 x + 8
r=
   3     2
1 x + 3 x + 5 x + 7
(True, '')

p(x)= [4.+9.65685425e+00j 4.+4.00000000e+00j 4.+1.65685425e+00j
 4.+4.89858720e-16j 4.-1.65685425e+00j 4.-4.00000000e+00j
 4.-9.65685425e+00j]
(True, '')


In [12]:
# Efficient polynomial evaluation at the nth values of (n+1) roots of the unit using FFT type recursion, when (n+1) is a power of 2
# In actual fast multiplication, a recurse algorithm should probably not be used
# x1 contains all the roots ending by 1 to simplify code, but polynomial is not evaluated for this value
def epex(p, x1):
    # If polynomial is small, direct evaluation
    if len(p.c)<=3:
        #print("$1: ", p.c)
        return p(x1[:-1])
    
    # Recursive evaluation, posing p(x) = x*r(x²)+q(x²)
    # q = polynomial(coefficients of even powers of p), contains constant coefficient (power 0)
    # r = polynomial(coefficients of odd powers of p)
    # x² = x[0, 2, 4...] modulo n since elements of x are unity roots
    if len(p.c)%2==1:
        q = np.poly1d(p.c[::2])
        r = np.poly1d(p.c[1::2])
    else:
        q = np.poly1d(p.c[1::2])
        r = np.poly1d(p.c[::2])
    x2 = np.append(x1[1::2], x1[1::2])
    return x1[:-1]*epex(r, x2) + epex(q, x2)

In [13]:
# Test 1
p = np.poly1d([1,2,3,4,5,6,7,8])
l = math.ceil(math.log(len(p.c))/math.log(2))
n = 2**l-1
x1 = nthRootsOfUnity1(n+1)
y = epex(p, x1)
print(y)

[4.+9.65685425e+00j 4.+4.00000000e+00j 4.+1.65685425e+00j
 4.+9.79717439e-16j 4.-1.65685425e+00j 4.-4.00000000e+00j
 4.-9.65685425e+00j]


In [14]:
# Another test, to make sure epex returns a result identical to direct evaluation (y2)
p = np.poly1d([1,2,3,4])
l = math.ceil(math.log(len(p.c))/math.log(2))
n = 2**l-1
x1 = nthRootsOfUnity1(n+1)
y1 = epex(p, x1)
y2 = p(x1[:-1])
print(y1)
print(y2)

print(cmplc(y1, y2))

[2.+2.0000000e+00j 2.+2.4492936e-16j 2.-2.0000000e+00j]
[2.+2.0000000e+00j 2.+2.4492936e-16j 2.-2.0000000e+00j]
(True, '')


In [15]:
# Helper number -> polynomial
def numtopoly(n):
    return np.poly1d([int(c) for c in str(n)])
    
print(numtopoly(123))

2
1 x + 2 x + 3


In [16]:
# Another test
p = numtopoly(12345)
l = 4
n = 2**l-1
x1 = nthRootsOfUnity1(n+1)
y1 = epex(p, x1)
y2 = p(x1[:-1])
print(y1)
print(y2)

print(cmplc(y1, y2))

[11.58220534+6.49981314e+00j  5.41421356+7.24264069e+00j
  2.56165432+4.05147161e+00j  3.        +2.00000000e+00j
  3.19570499+1.80883092e+00j  2.58578644+1.24264069e+00j
  2.66043535+2.57172451e-01j  3.        -1.97215226e-31j
  2.66043535-2.57172451e-01j  2.58578644-1.24264069e+00j
  3.19570499-1.80883092e+00j  3.        -2.00000000e+00j
  2.56165432-4.05147161e+00j  5.41421356-7.24264069e+00j
 11.58220534-6.49981314e+00j]
[11.58220534+6.49981314j  5.41421356+7.24264069j  2.56165432+4.05147161j
  3.        +2.j          3.19570499+1.80883092j  2.58578644+1.24264069j
  2.66043535+0.25717245j  3.        +0.j          2.66043535-0.25717245j
  2.58578644-1.24264069j  3.19570499-1.80883092j  3.        -2.j
  2.56165432-4.05147161j  5.41421356-7.24264069j 11.58220534-6.49981314j]
(True, '')


In [17]:
from numpy.linalg import inv

# Personal implementation of Langrange interpolation since scipy.interpolate.lgrange is unstable according to documentation
# and can't be user on more than 20 values. Indeed doesn't give correct results when mult result is over 32th root of unit.
# Use vandermonde matrix to interpolate, we have Y = M x P where M is vandemonde(X) and P a vector form of the polynomial
# so P = inv(M) x Y, returned as a poly1d
# Implemented this way it has a high cost of inverting a matrix
def myLagrange(x, y):
    mint = inv(np.vander(x))
    return np.poly1d(mint.dot(y))

x = np.array([2,3,4])
y = x**2+3*x+2
print(lagrange(x, y))
print(myLagrange(x, y))

2
1 x + 3 x + 2
   2
1 x + 3 x + 2


In [18]:
# Final application, multiply two large numbers

def lmul(a, b):
    pa = numtopoly(a)
    pb = numtopoly(b)
    #print(pa)
    #print(pb)
    l = math.ceil(math.log(len(pa.c)+len(pb.c))/math.log(2))
    n = 2**l-1
    x = nthRootsOfUnity1(n+1)
    #print("l=",l)
    #print("n=", n)

    ea = epex(pa, x)
    #ea2 = pa(x[:-1])
    #print(cmplc(ea, ea2))
    eb = epex(pb, x)
    #eb2 = pb(x[:-1])
    #print(cmplc(eb, eb2))

    y = ea*eb
    pm = myLagrange(x[:-1], y)
    m = polytonum(pm)
    return m

"""
# This test fails with scipy.interpolate.lgrange
a = 12345678901234567
b = 98765432109876543
m1 = lmul(a,b)
m2 = a*b
print(m1)
print(m2)
print(m1==m2)
"""

# Many tests, up to 50x50 digits
def tmul():
    sa = "1234567890"*5
    sb = "9876543210"*5

    for i in range(len(sa)):
        a = int(sa[:i+1])
        b = int(sb[:i+1])
        m = lmul(a, b)
        print(i+1, a, b, a*b==m)

tmul()

1 1 9 True
2 12 98 True
3 123 987 True
4 1234 9876 True
5 12345 98765 True
6 123456 987654 True
7 1234567 9876543 True
8 12345678 98765432 True
9 123456789 987654321 True
10 1234567890 9876543210 True
11 12345678901 98765432109 True
12 123456789012 987654321098 True
13 1234567890123 9876543210987 True
14 12345678901234 98765432109876 True
15 123456789012345 987654321098765 True
16 1234567890123456 9876543210987654 True
17 12345678901234567 98765432109876543 True
18 123456789012345678 987654321098765432 True
19 1234567890123456789 9876543210987654321 True
20 12345678901234567890 98765432109876543210 True
21 123456789012345678901 987654321098765432109 True
22 1234567890123456789012 9876543210987654321098 True
23 12345678901234567890123 98765432109876543210987 True
24 123456789012345678901234 987654321098765432109876 True
25 1234567890123456789012345 9876543210987654321098765 True
26 12345678901234567890123456 98765432109876543210987654 True
27 123456789012345678901234567 9876543210987654